In [ ]:
%%time
import numpy as np
import pandas as pd
import warnings
#from MyUtils import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [ ]:
import time

ts = str(int(time.time()))

In [ ]:
# Select the time-to-event time
def selected_time(selected_time, time, event):
    if (time > selected_time): return 0
    else:
        if (event == 1.0): return 1
        else: return None

In [ ]:
def numerical_stages(stage):
    if stage=='Stage I' or stage=='Stage IA':
        numerical_stage=0.0
    if stage=='Stage IB':
        numerical_stage=5.0
    if stage=='Stage II' or stage=='Stage IIA':
        numerical_stage=10.0
    if stage=='Stage IIB':
        numerical_stage=15.0
    if stage=='Stage III' or stage=='Stage IIIA':
        numerical_stage=20.0
    if stage=='Stage IIIB':
        numerical_stage=25.0
    if stage=='Stage IV':
        numerical_stage=30.0
    return numerical_stage

In [ ]:
survival_metric='OS'
survival_time='OS.time'
time=1800
model_save_path='./saved_models/lung_model_cnn_{}_{}_{}.h5'.format(survival_metric,time,ts)

In [ ]:
%%time
Y_type_L=pd.read_hdf('data/Preprocessed_Data/Lung.h5',key='sample_type')
Y_clinical_L=pd.read_hdf('data/Preprocessed_Data/Lung.h5',key='clinical')

In [ ]:
Y_clinical_L.head()

In [ ]:
Y_clinical_L=Y_clinical_L.loc[Y_type_L.pathology_status=='Tumor']
Y_clinical_L.dropna(subset=[survival_metric, survival_time], inplace=True)
Y_clinical_L.dropna(subset=['age_at_initial_pathologic_diagnosis'],inplace=True)
Y_clinical_L.dropna(subset=['ajcc_pathologic_tumor_stage'],inplace=True)
Y_clinical_L=Y_clinical_L[~Y_clinical_L['ajcc_pathologic_tumor_stage'].isin(['[Discrepancy]'])]

In [ ]:
Y_clinical_L['ajcc_pathologic_tumor_stage'].value_counts()

In [ ]:
num_stage = Y_clinical_L[['ajcc_pathologic_tumor_stage']].apply(lambda row: numerical_stages(row['ajcc_pathologic_tumor_stage']), axis=1)

In [ ]:
num_stage

In [ ]:
Y_clinical_categorical_L = Y_clinical_L[[survival_metric, survival_time]].apply(lambda row: selected_time(time, row[survival_time], row[survival_metric]), axis=1)
Y_clinical_categorical_L.dropna(inplace=True)
Y_clinical_categorical_L

In [ ]:
age=Y_clinical_L['age_at_initial_pathologic_diagnosis']

In [ ]:
age=np.array(age.loc[[s for s in Y_clinical_categorical_L.index]])

In [ ]:
age=age.reshape(age.shape[0],1)

In [ ]:
age.shape

In [ ]:
stage=np.array(num_stage.loc[[s for s in Y_clinical_categorical_L.index]])
stage=stage.reshape(stage.shape[0],1)

In [ ]:
Y_clinical_categorical_L.value_counts(normalize=True)

In [ ]:
%%time
#读取图像数据
dir_name_p = 'images/genes_selected/log2tpm_MinMax/lung_with_keggpathway_1k/'
dir_name_f = 'images/genes_selected/log2tpm_MinMax/lung_with_function_1k/'
#dir_name = 'images/genes_selected/log2tpm_MinMax/lung_no_pathway/'
X_p = np.array([np.load(dir_name_p+s+".npy") for s in Y_clinical_categorical_L.index]) 
X_f = np.array([np.load(dir_name_f+s+".npy") for s in Y_clinical_categorical_L.index]) 

In [ ]:
print(X_p.shape)
print(X_f.shape)

In [ ]:
%%time
from sklearn.preprocessing import LabelEncoder
y_L = LabelEncoder().fit_transform(Y_clinical_categorical_L)
y_L=y_L.reshape(y_L.shape[0],1)

In [ ]:
%%time
#划分训练集测试集
from sklearn.model_selection import train_test_split
X_p_train,X_p_test,X_f_train,X_f_test,age_train,age_test,stage_train,stage_test,Y_clinical_categorical_L_train,Y_clinical_categorical_L_test,y_train,y_test=train_test_split(X_p,X_f,age,stage,Y_clinical_categorical_L,y_L,test_size=0.2,random_state=126,stratify=y_L)
print(X_p_train.shape)
print(X_p_test.shape)
print(X_f_train.shape)
print(X_f_test.shape)
print(age_train.shape)
print(age_test.shape)
print(stage_train.shape)
print(stage_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
%%time
X_random_p_train=X_p_train.reshape(X_p_train.shape[0],26*26)
X_random_f_train=X_f_train.reshape(X_f_train.shape[0],27*27)
age_random_train=age_train
stage_random_train=stage_train
X_i_train=[[i] for i in range(len(y_train))]
#X_smote.shape
from imblearn.over_sampling import SMOTE as os


X_ri_train, y_r_train = os(random_state=321).fit_resample(X_i_train,y_train)
X_p_r_train=X_random_p_train[[i[0] for i in X_ri_train]]
X_f_r_train=X_random_f_train[[i[0] for i in X_ri_train]]
age_r_train=age_random_train[[i[0] for i in X_ri_train]]
stage_r_train=stage_random_train[[i[0] for i in X_ri_train]]

X_p_r_train=X_p_r_train.reshape(X_p_r_train.shape[0],26,26,1)
X_f_r_train=X_f_r_train.reshape(X_f_r_train.shape[0],27,27,1)
y_r_train=y_r_train.reshape(y_r_train.shape[0],1)
print(X_p_r_train.shape)
print(X_f_r_train.shape)
print(age_r_train.shape)
print(stage_r_train.shape)
print(y_r_train.shape)

In [ ]:
import keras_tuner as kt
def build_dual_inputs_cnn(hp):
    
    function_input=keras.layers.Input(shape=(27,27,1))
    pathway_input=keras.layers.Input(shape=(26,26,1))
    #age_input=keras.layers.Input(shape=(1,))
    #stage_input=keras.layers.Input(shape=(1,))
    
    f_conv_1=layers.Conv2D(hp.Choice('filters1',[32,40,48,56,64]), 
                           kernel_size=(3,3), 
                           strides=1,
                           padding='same',
                           use_bias=False, 
                           activation='relu',
                           kernel_initializer='he_uniform', 
                           name='f_conv_1',
                           )(function_input)
    
    f_pooling1=layers.MaxPooling2D(pool_size=(3,3),
                         strides=2, 
                         padding='same')(f_conv_1)
    
    f_bn1=layers.BatchNormalization()(f_pooling1)
    
    f_conv_2=layers.Conv2D(hp.Choice('filters2',[80,96,112,128]), 
                           kernel_size=(3,3), 
                           strides=1,
                           padding='same',
                           use_bias=False, 
                           activation='relu',
                           kernel_initializer='he_uniform', 
                           name='f_conv_2',
                           )(f_bn1)
    
    #f_padding_1=layers.ZeroPadding2D(3)(f_conv_1)
    
    f_pooling2=layers.MaxPooling2D(pool_size=(3,3),
                         strides=2, 
                         padding='same')(f_conv_2) 
    
    f_bn2=layers.BatchNormalization()(f_pooling2)
    
    f_fn1=layers.Dense(hp.Choice('units_f',[128,144,192,256]),activation='relu')(f_bn2)
    
    f_fn1=layers.Dropout(hp.Choice('drf',[0.1,0.2,0.3]))(f_fn1)
    
    p_conv_1=layers.Conv2D(hp.Choice('filters3',[32,40,48,56,64]), 
                           kernel_size=(3,3), 
                           strides=1,
                           padding='same',
                           use_bias=False, 
                           activation='relu',
                           kernel_initializer='he_uniform', 
                           name='p_conv_1',
                           )(pathway_input)
    
    p_pooling1=layers.MaxPooling2D(pool_size=(3,3),
                         strides=2, 
                         padding='same')(p_conv_1)
    
    p_bn1=layers.BatchNormalization()(p_pooling1)
    
    p_conv_2=layers.Conv2D(hp.Choice('filters4',[80,96,112,128]), 
                           kernel_size=(3,3), 
                           strides=1,
                           padding='same',
                           use_bias=False, 
                           activation='relu',
                           kernel_initializer='he_uniform', 
                           name='p_conv_2',
                           )(p_bn1)
    
    #f_padding_1=layers.ZeroPadding2D(3)(f_conv_1)
    
    p_pooling2=layers.MaxPooling2D(pool_size=(3,3),
                         strides=2, 
                         padding='same')(p_conv_2)
    
    p_bn2=layers.BatchNormalization()(p_pooling2)
    
    p_fn1=layers.Dense(hp.Choice('units_p',[128,144,192,256]),activation='relu')(p_bn2)
    
    p_fn1=layers.Dropout(hp.Choice('drp',[0.1,0.2,0.3]))(p_fn1)
    
    x=layers.concatenate([f_fn1,p_fn1])
    
    x=layers.Flatten()(x)
    
    #x=layers.concatenate([x,age_input])
    
    #x=layers.GlobalAveragePooling2D()(x)
    
    #x=layers.Dense(hp.Choice('units1',[32,64,96]),activation='relu')(x)
    
    #x=layers.Dropout(hp.Choice('dr1',[0.1,0.2,0.3]))(x)
    
    x=layers.Dense(hp.Choice('units2',[64,128,144,192,256]),activation='relu')(x)
    
    x=layers.Dropout(hp.Choice('dr2',[0.3,0.4,0.5]))(x)
    
    #x=layers.Flatten()(x)
    
    #x=layers.concatenate([x,age_input,stage_input])
    
    x=layers.Dense(hp.Choice('units3',[32,64,128]),activation='relu')(x)
    
    x=layers.Dropout(hp.Choice('dr3',[0.3,0.4,0.5]))(x)
    
    output=layers.Dense(1,activation='sigmoid')(x)
    
    model=keras.Model(inputs=[function_input,pathway_input],outputs=output)
    
    optimizer = tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001)

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(
            from_logits=False, label_smoothing=0.1
        ),
        metrics=[
#            keras.metrics.BinaryAccuracy(name="accuracy"),
#            keras.metrics.BinaryAccuracy(
#                name="binary_accuracy", dtype=None, threshold=0.5
#            ),
            keras.metrics.AUC(name='auc',from_logits=False),
#            keras.metrics.Precision(name='precision'),
#            keras.metrics.Recall(name='recall'),  
#            tfa.metrics.F1Score(num_classes,threshold=0.5,name='f1_score')
        ],
    )
    return model

In [ ]:
tuner = kt.BayesianOptimization(
        build_dual_inputs_cnn,
        objective=kt.Objective("val_auc", direction="max"),
        max_trials=100,
        overwrite=True)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_auc',mode='max', patience=6)
tuner.search([X_f_r_train,X_p_r_train], y_r_train,epochs=50, validation_data=([X_f_test,X_p_test], y_test),callbacks=[stop_early])
best_model = tuner.get_best_models()[0]

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

In [ ]:
model=best_model
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(best_model, to_file='model1.png')

In [ ]:
from sklearn.metrics import roc_curve
import sklearn.metrics
def train_model(model):
    #set dynamic learning rate
    initial_learning_rate = 0.0001
    '''
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=100000,
        decay_rate=0.96,
        staircase=True)
    '''
    optimizer = tfa.optimizers.AdamW(learning_rate=0.00001, weight_decay=0.0001)
    #optimizer=keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(
            from_logits=False, label_smoothing=0.1
        ),
        metrics=[
#            keras.metrics.BinaryAccuracy(name="accuracy"),
#            keras.metrics.BinaryAccuracy(
#                name="binary_accuracy", dtype=None, threshold=0.5
#            ),
            keras.metrics.AUC(name='auc',from_logits=False,num_thresholds=500),
#            keras.metrics.Precision(name='precision'),
#            keras.metrics.Recall(name='recall'),  
#            tfa.metrics.F1Score(num_classes,threshold=0.5,name='f1_score')
        ],
    )

    checkpoint_filepath = "./tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_auc",
        mode='max',
        save_best_only=True,
        save_weights_only=True,
    )
    
    early_stop = keras.callbacks.EarlyStopping(monitor='val_auc',mode='max', patience=10)

    history = model.fit(
        x=([X_f_r_train,X_p_r_train,age_r_train,stage_r_train]),
        y=y_r_train,
        batch_size=32,
        epochs=150,
        validation_data=([X_f_test,X_p_test,age_test,stage_test], y_test),
        callbacks=[checkpoint_callback,early_stop],
    )

    model.load_weights(checkpoint_filepath)
    y_pred=model.predict([X_f_test,X_p_test,age_test,stage_test])
    auc=sklearn.metrics.roc_auc_score(y_test,y_pred)
    Y_clinical_categorical_pred=pd.DataFrame(Y_clinical_categorical_L_test)
    Y_clinical_categorical_pred['prediction']=y_pred
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    J = tpr - fpr
    index_max = np.argmax(J)
    best_thresh = thresholds[index_max]
    
    tp=Y_clinical_categorical_pred[(Y_clinical_categorical_pred[0]==1.0) & (Y_clinical_categorical_pred['prediction']>=best_thresh)].shape[0]
    fp=Y_clinical_categorical_pred[(Y_clinical_categorical_pred[0]==1.0) & (Y_clinical_categorical_pred['prediction']<best_thresh)].shape[0]
    tn=Y_clinical_categorical_pred[(Y_clinical_categorical_pred[0]==0.0) & (Y_clinical_categorical_pred['prediction']<=best_thresh)].shape[0]
    fn=Y_clinical_categorical_pred[(Y_clinical_categorical_pred[0]==0.0) & (Y_clinical_categorical_pred['prediction']>best_thresh)].shape[0]
    accuracy=(tp+tn)/(tp+fp+tn+fn)
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1_score=2*(precision*recall)/(precision+recall)
    
    #    _, accuracy, binary_accuracy,auc_roc = model.evaluate(X_test, y_test)
    #_, accuracy,auc,precision,recall = model.evaluate([X_f_test,X_p_test,age_test,stage_test], y_test)
   
    print('Best Threshold=%f' % (best_thresh))
    print(f"Test accuracy: {round(accuracy,4)}")
    print(f"Test AUC: {round(auc, 4)}")
    print(f"Test precision: {round(precision, 4)}")
    print(f"Test recall: {round(recall, 4)}")
    print(f"Test f1_score: {round(f1_score,4)}")
    
    return history 

In [ ]:
model=best_model
history=train_model(model)

In [ ]:
%%time
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE as os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
import sklearn.metrics
accuracy_list=[]
auc_list=[]
precision_list=[]
recall_list=[]
f1_score_list=[]
init_random_state=255
for i in range(0,50):
    X_p_train,X_p_test,X_f_train,X_f_test,age_train,age_test,stage_train,stage_test,Y_clinical_categorical_L_train,Y_clinical_categorical_L_test,y_train,y_test=train_test_split(X_p,X_f,age,stage,Y_clinical_categorical_L,y_L,test_size=0.2,random_state=init_random_state+i,stratify=y_L)
    print('=============================================================================================================')
    print('* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *')
    print(' * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ')
    print('* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *')
    print(' * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ')
    print('* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *')
    print('=============================================================================================================')
    print('current random state is: {}'.format(init_random_state+i))
    print('This is the {}th fold'.format(i+1))
    
    X_random_p_train=X_p_train.reshape(X_p_train.shape[0],26*26)
    X_random_f_train=X_f_train.reshape(X_f_train.shape[0],27*27)
    age_random_train=age_train
    stage_random_train=stage_train
    X_i_train=[[i] for i in range(len(y_train))]
    X_ri_train, y_r_train = os(random_state=321+i).fit_resample(X_i_train,y_train)
    X_p_r_train=X_random_p_train[[i[0] for i in X_ri_train]]
    X_f_r_train=X_random_f_train[[i[0] for i in X_ri_train]]
    age_r_train=age_random_train[[i[0] for i in X_ri_train]]
    stage_r_train=stage_random_train[[i[0] for i in X_ri_train]]

    X_p_r_train=X_p_r_train.reshape(X_p_r_train.shape[0],26,26,1)
    X_f_r_train=X_f_r_train.reshape(X_f_r_train.shape[0],27,27,1)
    y_r_train=y_r_train.reshape(y_r_train.shape[0],1)
    
    model=build_dual_inputs_cnn(best_hyperparameters)
    
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10,
    decay_rate=0.99)

    
    optimizer = tfa.optimizers.AdamW(learning_rate=lr_schedule, weight_decay=0.0001)
    #optimizer=keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(
            from_logits=False, label_smoothing=0.1
        ),
        metrics=[
#            keras.metrics.BinaryAccuracy(name="accuracy"),
#            keras.metrics.BinaryAccuracy(
#                name="binary_accuracy", dtype=None, threshold=0.5
#            ),
            keras.metrics.AUC(name='auc',from_logits=False,num_thresholds=500),
#            keras.metrics.Precision(name='precision'),
#            keras.metrics.Recall(name='recall'),  
#            tfa.metrics.F1Score(num_classes,threshold=0.5,name='f1_score')
        ],
    )

    checkpoint_filepath = "./tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_auc",
        mode='max',
        save_best_only=True,
        save_weights_only=True,
    )
    
    early_stop = keras.callbacks.EarlyStopping(monitor='val_auc',mode='max', patience=30)

    history = model.fit(
        x=([X_f_r_train,X_p_r_train]),
        y=y_r_train,
        batch_size=32,
        epochs=150,
        validation_data=([X_f_test,X_p_test], y_test),
        callbacks=[checkpoint_callback,early_stop],
    )

    model.load_weights(checkpoint_filepath)
#    _, accuracy, binary_accuracy,auc_roc = model.evaluate(X_test, y_test)
    #_, accuracy,auc,precision,recall = model.evaluate([X_f_test,X_p_test,age_test,stage_test], y_test)
    y_pred=model.predict([X_f_test,X_p_test])
    auc=sklearn.metrics.roc_auc_score(y_test,y_pred)
    Y_clinical_categorical_pred=pd.DataFrame(Y_clinical_categorical_L_test)
    Y_clinical_categorical_pred['prediction']=y_pred
    
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    J = tpr - fpr
    index_max = np.argmax(J)
    best_thresh = thresholds[index_max]
    
    tp=Y_clinical_categorical_pred[(Y_clinical_categorical_pred[0]==1.0) & (Y_clinical_categorical_pred['prediction']>=best_thresh)].shape[0]
    fp=Y_clinical_categorical_pred[(Y_clinical_categorical_pred[0]==1.0) & (Y_clinical_categorical_pred['prediction']<best_thresh)].shape[0]
    tn=Y_clinical_categorical_pred[(Y_clinical_categorical_pred[0]==0.0) & (Y_clinical_categorical_pred['prediction']<=best_thresh)].shape[0]
    fn=Y_clinical_categorical_pred[(Y_clinical_categorical_pred[0]==0.0) & (Y_clinical_categorical_pred['prediction']>best_thresh)].shape[0]
    
    accuracy=(tp+tn)/(tp+fp+tn+fn)
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1_score=2*(precision*recall)/(precision+recall) 
    
   
    #print(f"Test accuracy: {round(accuracy * 100, 2)}%")
#    print(f"Test binary accuracy: {round(binary_accuracy * 100, 2)}%")
    #print(f"Test AUC: {round(auc, 4)}")
    #print(f"Test precision: {round(precision * 100, 2)}%")
    #print(f"Test recall: {round(recall * 100, 2)}%")
    #print(f"Test f1_score: {round(f1_score*100,2)}%")
    accuracy_list.append(accuracy)
    auc_list.append(auc)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_score_list.append(f1_score)
    
print('mean accuracy={}±{}'.format(np.mean(accuracy_list),np.std(accuracy_list)))
print('mean auc={}±{}'.format(np.mean(auc_list),np.std(auc_list)))
print('mean precision={}±{}'.format(np.mean(precision_list),np.std(precision_list)))
print('mean recall={}±{}'.format(np.mean(recall_list),np.std(recall_list)))
print('mean f1_score={}±{}'.format(np.mean(f1_score_list),np.std(f1_score_list)))

In [ ]:
f1_score_list

In [ ]:
import sklearn.metrics
y_pred=model.predict([X_f_test,X_p_test,age_test,stage_test])
auc=sklearn.metrics.roc_auc_score(y_test,y_pred)
print(auc)

In [ ]:
y_pred

In [ ]:
f1_score_list

In [ ]:
predictions=model.predict([X_f_test,X_p_test,age_test,stage_test])

In [ ]:
#perform the youden's J statistics best threshold selection
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
J = tpr - fpr
index_max = np.argmax(J)
best_thresh = thresholds[index_max]
print('Best Threshold=%f' % (best_thresh))

In [ ]:
Y_clinical_categorical_pred=pd.DataFrame(Y_clinical_categorical_L_test)

In [ ]:
Y_clinical_categorical_pred['prediction']=predictions

In [ ]:
Y_clinical_categorical_pred[0]

In [ ]:
tp=Y_clinical_categorical_pred[(Y_clinical_categorical_pred[0]==1.0) & (Y_clinical_categorical_pred['prediction']>=best_thresh)].shape[0]
fp=Y_clinical_categorical_pred[(Y_clinical_categorical_pred[0]==1.0) & (Y_clinical_categorical_pred['prediction']<best_thresh)].shape[0]
tn=Y_clinical_categorical_pred[(Y_clinical_categorical_pred[0]==0.0) & (Y_clinical_categorical_pred['prediction']<=best_thresh)].shape[0]
fn=Y_clinical_categorical_pred[(Y_clinical_categorical_pred[0]==0.0) & (Y_clinical_categorical_pred['prediction']>best_thresh)].shape[0]


In [ ]:
real_acc=(tp+tn)/95
real_precision=tp/(tp+fp)
real_recall=tp/(tp+fn)
real_f1_score=2*(real_precision*real_recall)/(real_precision+real_recall)
real_acc,real_precision,real_recall,real_f1_score

In [ ]:
pred_positive=Y_clinical_categorical_pred.loc[Y_clinical_categorical_pred.prediction>=0.626445]

In [ ]:
pred_negative=Y_clinical_categorical_pred.loc[Y_clinical_categorical_pred.prediction<0.626445]

In [ ]:
clinical_positive=Y_clinical_L.loc[[s for s in pred_positive.index]]

In [ ]:
clinical_negative=Y_clinical_L.loc[[s for s in pred_negative.index]]

In [ ]:
clinical_positive

In [ ]:
clinical_negative

In [ ]:
from lifelines import KaplanMeierFitter
from matplotlib import pyplot as plt

timeline=[]
for i in range(0,1830):
    timeline.append(i)

In [ ]:
from lifelines.statistics import logrank_test
from lifelines.utils import restricted_mean_survival_time
from lifelines.plotting import rmst_plot

T_p=clinical_positive['OS.time']
E_p=clinical_positive['OS']
kmf_positive = KaplanMeierFitter()
kmf_positive.fit(durations=T_p, event_observed=E_p,timeline=timeline,label='Predicted Positive')
ax=kmf_positive.plot()

T_n=clinical_negative['OS.time']
E_n=clinical_negative['OS']
kmf_negative=KaplanMeierFitter()
kmf_negative.fit(durations=T_n, event_observed=E_n,timeline=timeline,label='Predicted Negative')
ax=kmf_negative.plot(ax=ax)

results = logrank_test(T_p, T_n, event_observed_A=E_p, event_observed_B=E_n)
'''
rmst_positive= restricted_mean_survival_time(kmf_positive, t=900)
rmst_negative= restricted_mean_survival_time(kmf_negative, t=900)

ax = plt.subplot(311)
rmst_plot(kmf_negative, t=900, ax=ax)


ax = plt.subplot(312)
rmst_plot(kmf_positive, t=900, ax=ax)


ax = plt.subplot(313)
rmst_plot(kmf_negative, model2=kmf_positive, t=900, ax=ax)
'''
plt.legend(loc='lower left', bbox_to_anchor=(0.01,0.01),ncol=1,fancybox=False,shadow=False)
plt.title('SiameseSurvNet p-value: {}'.format(results.p_value))

In [ ]:
ax.get_figure().savefig("figures/SiameseSurvNet_kpcurve.svg",dpi=600,format='svg')